# Ionosphere Database

Эти радиолокационные данные были собраны системой в Гусином заливе, Лабрадор. Эта система состоит из фазированной решетки из 16 высокочастотных антенн с общей передаваемой мощностью порядка 6,4 киловатт. Более подробную информацию смотрите в документе. Мишенями были свободные электроны в ионосфере. "Хорошие" (Good) радиолокационные сигналы - это те, которые показывают признаки некоторого типа структуры в ионосфере. "Плохие" (Bad) - это те, которые этого не делают; их сигналы проходят через ионосферу.

Принятые сигналы обрабатывались с использованием автокорреляционной функции, аргументами которой являются время импульса и номер импульса. Для системы Goose Bay было 17 номеров импульсов. Экземпляры в этой базе данных описываются 2 атрибутами на номер импульса, соответствующими комплексным значениям, возвращаемым функцией в результате комплексного электромагнитного сигнала.

## Additional Variable Information

-- Все 34 являются непрерывными
-- 35-й атрибут является либо "хорошим", либо "плохим" в соответствии с приведенным выше определением.  Это задача бинарной классификации.

# Код

Импорт необходимых библиотек и загрузка датасета.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

In [ ]:
with open('ionosphere.names') as f:
 df_info = f.read()

f.close()
print(df_info)

1. Title: Johns Hopkins University Ionosphere database

2. Source Information:
   -- Donor: Vince Sigillito (vgs@aplcen.apl.jhu.edu)
   -- Date: 1989
   -- Source: Space Physics Group
              Applied Physics Laboratory
              Johns Hopkins University
              Johns Hopkins Road
              Laurel, MD 20723 

3. Past Usage:
   -- Sigillito, V. G., Wing, S. P., Hutton, L. V., \& Baker, K. B. (1989).
      Classification of radar returns from the ionosphere using neural 
      networks. Johns Hopkins APL Technical Digest, 10, 262-266.

      They investigated using backprop and the perceptron training algorithm
      on this database.  Using the first 200 instances for training, which
      were carefully split almost 50% positive and 50% negative, they found
      that a "linear" perceptron attained 90.7%, a "non-linear" perceptron
      attained 92%, and backprop an average of over 96% accuracy on the 
      remaining 150 test instances, consisting of 123 "good" and 

На основе данных из описания датасета сформируем наименования колонок будущего датафрейма.

In [ ]:
columns = []

for i in range(35):
  if i < 34:
    columns.append('Attribute' + str(i+1))
  else:
    columns.append('Class')

columns

['Attribute1',
 'Attribute2',
 'Attribute3',
 'Attribute4',
 'Attribute5',
 'Attribute6',
 'Attribute7',
 'Attribute8',
 'Attribute9',
 'Attribute10',
 'Attribute11',
 'Attribute12',
 'Attribute13',
 'Attribute14',
 'Attribute15',
 'Attribute16',
 'Attribute17',
 'Attribute18',
 'Attribute19',
 'Attribute20',
 'Attribute21',
 'Attribute22',
 'Attribute23',
 'Attribute24',
 'Attribute25',
 'Attribute26',
 'Attribute27',
 'Attribute28',
 'Attribute29',
 'Attribute30',
 'Attribute31',
 'Attribute32',
 'Attribute33',
 'Attribute34',
 'Class']

In [ ]:
columns_dict = {idx: name for idx, name in enumerate(columns)}
columns_dict

{0: 'Attribute1',
 1: 'Attribute2',
 2: 'Attribute3',
 3: 'Attribute4',
 4: 'Attribute5',
 5: 'Attribute6',
 6: 'Attribute7',
 7: 'Attribute8',
 8: 'Attribute9',
 9: 'Attribute10',
 10: 'Attribute11',
 11: 'Attribute12',
 12: 'Attribute13',
 13: 'Attribute14',
 14: 'Attribute15',
 15: 'Attribute16',
 16: 'Attribute17',
 17: 'Attribute18',
 18: 'Attribute19',
 19: 'Attribute20',
 20: 'Attribute21',
 21: 'Attribute22',
 22: 'Attribute23',
 23: 'Attribute24',
 24: 'Attribute25',
 25: 'Attribute26',
 26: 'Attribute27',
 27: 'Attribute28',
 28: 'Attribute29',
 29: 'Attribute30',
 30: 'Attribute31',
 31: 'Attribute32',
 32: 'Attribute33',
 33: 'Attribute34',
 34: 'Class'}

Считываем сами данные из соответствующего файла.

In [ ]:
df_orig = pd.read_csv('ionosphere.data', header = None)
df_orig

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,...,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,g
1,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,...,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,b
2,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,...,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,g
3,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,...,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,b
4,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,...,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,1,0,0.83508,0.08298,0.73739,-0.14706,0.84349,-0.05567,0.90441,-0.04622,...,-0.04202,0.83479,0.00123,1.00000,0.12815,0.86660,-0.10714,0.90546,-0.04307,g
347,1,0,0.95113,0.00419,0.95183,-0.02723,0.93438,-0.01920,0.94590,0.01606,...,0.01361,0.93522,0.04925,0.93159,0.08168,0.94066,-0.00035,0.91483,0.04712,g
348,1,0,0.94701,-0.00034,0.93207,-0.03227,0.95177,-0.03431,0.95584,0.02446,...,0.03193,0.92489,0.02542,0.92120,0.02242,0.92459,0.00442,0.92697,-0.00577,g
349,1,0,0.90608,-0.01657,0.98122,-0.01989,0.95691,-0.03646,0.85746,0.00110,...,-0.02099,0.89147,-0.07760,0.82983,-0.17238,0.96022,-0.03757,0.87403,-0.16243,g


Копируем датафрейм, чтобы изначальный оставался без изменения, а работоать можно было бы с копией без последствий.

In [ ]:
df_orig.rename(columns=columns_dict, inplace=True)
df = df_orig.copy()

df

,Attribute1,Attribute2,Attribute3,Attribute4,Attribute5,Attribute6,Attribute7,Attribute8,Attribute9,Attribute10,...,Attribute26,Attribute27,Attribute28,Attribute29,Attribute30,Attribute31,Attribute32,Attribute33,Attribute34,Class
0,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,...,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,g
1,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,...,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,b
2,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,...,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,g
3,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,...,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,b
4,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,...,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,1,0,0.83508,0.08298,0.73739,-0.14706,0.84349,-0.05567,0.90441,-0.04622,...,-0.04202,0.83479,0.00123,1.00000,0.12815,0.86660,-0.10714,0.90546,-0.04307,g
347,1,0,0.95113,0.00419,0.95183,-0.02723,0.93438,-0.01920,0.94590,0.01606,...,0.01361,0.93522,0.04925,0.93159,0.08168,0.94066,-0.00035,0.91483,0.04712,g
348,1,0,0.94701,-0.00034,0.93207,-0.03227,0.95177,-0.03431,0.95584,0.02446,...,0.03193,0.92489,0.02542,0.92120,0.02242,0.92459,0.00442,0.92697,-0.00577,g
349,1,0,0.90608,-0.01657,0.98122,-0.01989,0.95691,-0.03646,0.85746,0.00110,...,-0.02099,0.89147,-0.07760,0.82983,-0.17238,0.96022,-0.03757,0.87403,-0.16243,g


Ищем и удаляем дублирующиеся данные.

In [ ]:
df.duplicated().sum() # Дублирующиеся данные

1

In [ ]:
Dup_Rows = df[df.duplicated()]

print("\n\nПовторяющиеся строки : \n {}".format(Dup_Rows))



Повторяющиеся строки : 
      Attribute1  Attribute2  Attribute3  Attribute4  Attribute5  Attribute6  \
248           0           0         0.0         0.0         0.0         0.0   

     Attribute7  Attribute8  Attribute9  Attribute10  ...  Attribute26  \
248         0.0         0.0         0.0          0.0  ...          0.0   

     Attribute27  Attribute28  Attribute29  Attribute30  Attribute31  \
248          1.0         -1.0          0.0          0.0          0.0   

     Attribute32  Attribute33  Attribute34  Class  
248          0.0          0.0          0.0      b  

[1 rows x 35 columns]


In [ ]:
df = df.drop_duplicates(keep='first')
df.duplicated().sum()

0

Поиск отсутствующих значений, в данном дс таких нет.

In [ ]:
df.isna().sum() # Значения NOne

Attribute1     0
Attribute2     0
Attribute3     0
Attribute4     0
Attribute5     0
Attribute6     0
Attribute7     0
Attribute8     0
Attribute9     0
Attribute10    0
Attribute11    0
Attribute12    0
Attribute13    0
Attribute14    0
Attribute15    0
Attribute16    0
Attribute17    0
Attribute18    0
Attribute19    0
Attribute20    0
Attribute21    0
Attribute22    0
Attribute23    0
Attribute24    0
Attribute25    0
Attribute26    0
Attribute27    0
Attribute28    0
Attribute29    0
Attribute30    0
Attribute31    0
Attribute32    0
Attribute33    0
Attribute34    0
Class          0
dtype: int64

Последняя колонка дс Class имеет тип объекта, поскольку значения ячеек - строки, обозначающие принадлежности строк данных к классам Good или Bad (g, b). Заменим значения на 1 и 0 для g и b соответсвенно.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 350 entries, 0 to 350
Data columns (total 35 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Attribute1   350 non-null    int64  
 1   Attribute2   350 non-null    int64  
 2   Attribute3   350 non-null    float64
 3   Attribute4   350 non-null    float64
 4   Attribute5   350 non-null    float64
 5   Attribute6   350 non-null    float64
 6   Attribute7   350 non-null    float64
 7   Attribute8   350 non-null    float64
 8   Attribute9   350 non-null    float64
 9   Attribute10  350 non-null    float64
 10  Attribute11  350 non-null    float64
 11  Attribute12  350 non-null    float64
 12  Attribute13  350 non-null    float64
 13  Attribute14  350 non-null    float64
 14  Attribute15  350 non-null    float64
 15  Attribute16  350 non-null    float64
 16  Attribute17  350 non-null    float64
 17  Attribute18  350 non-null    float64
 18  Attribute19  350 non-null    float64
 19  Attribut

In [ ]:
# df['Class'].apply(lambda x: 1 if x == 'g' else 0)
df.loc[df['Class'] == 'g', 'Class'] = 1
df.loc[df['Class'] == 'b', 'Class'] = 0
df['Class'] = df['Class'].astype('int64')

<ipython-input-64-7beda4814810>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Class'] = df['Class'].astype('int64')


In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 350 entries, 0 to 350
Data columns (total 35 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Attribute1   350 non-null    int64  
 1   Attribute2   350 non-null    int64  
 2   Attribute3   350 non-null    float64
 3   Attribute4   350 non-null    float64
 4   Attribute5   350 non-null    float64
 5   Attribute6   350 non-null    float64
 6   Attribute7   350 non-null    float64
 7   Attribute8   350 non-null    float64
 8   Attribute9   350 non-null    float64
 9   Attribute10  350 non-null    float64
 10  Attribute11  350 non-null    float64
 11  Attribute12  350 non-null    float64
 12  Attribute13  350 non-null    float64
 13  Attribute14  350 non-null    float64
 14  Attribute15  350 non-null    float64
 15  Attribute16  350 non-null    float64
 16  Attribute17  350 non-null    float64
 17  Attribute18  350 non-null    float64
 18  Attribute19  350 non-null    float64
 19  Attribut

In [ ]:
df

,Attribute1,Attribute2,Attribute3,Attribute4,Attribute5,Attribute6,Attribute7,Attribute8,Attribute9,Attribute10,...,Attribute26,Attribute27,Attribute28,Attribute29,Attribute30,Attribute31,Attribute32,Attribute33,Attribute34,Class
0,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,...,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,1
1,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,...,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,0
2,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,...,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,1
3,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,...,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,0
4,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,...,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,1,0,0.83508,0.08298,0.73739,-0.14706,0.84349,-0.05567,0.90441,-0.04622,...,-0.04202,0.83479,0.00123,1.00000,0.12815,0.86660,-0.10714,0.90546,-0.04307,1
347,1,0,0.95113,0.00419,0.95183,-0.02723,0.93438,-0.01920,0.94590,0.01606,...,0.01361,0.93522,0.04925,0.93159,0.08168,0.94066,-0.00035,0.91483,0.04712,1
348,1,0,0.94701,-0.00034,0.93207,-0.03227,0.95177,-0.03431,0.95584,0.02446,...,0.03193,0.92489,0.02542,0.92120,0.02242,0.92459,0.00442,0.92697,-0.00577,1
349,1,0,0.90608,-0.01657,0.98122,-0.01989,0.95691,-0.03646,0.85746,0.00110,...,-0.02099,0.89147,-0.07760,0.82983,-0.17238,0.96022,-0.03757,0.87403,-0.16243,1


In [ ]:
X = np.array(df.iloc[:, 0:-1])
y = np.array(df[['Class']]).reshape(-1)

In [ ]:
X.shape, y.shape

((350, 34), (350,))

In [ ]:
def train_test_split(X, y, test_ratio=0.2, seed=None):
    """возвращает X_train, X_test, y_train, y_test"""
    assert X.shape[0] == y.shape[0], \
        "Размер X должен быть равен размеру y"
    assert 0.0 <= test_ratio <= 1.0, \
        "Неверное значение test_ratio"

    if seed:
        np.random.seed(seed)

    shuffled_indexes = np.random.permutation(len(X))

    test_size = int(len(X) * test_ratio)
    test_indexes = shuffled_indexes[:test_size]
    train_indexes = shuffled_indexes[test_size:]

    X_train = X[train_indexes]
    y_train = y[train_indexes]

    X_test = X[test_indexes]
    y_test = y[test_indexes]

    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 0.3)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((245, 34), (105, 34), (245,), (105,))

# Метод k ближайших соседей

Метод ближайших соседей (k Nearest Neighbors, или kNN) — тоже очень популярный метод классификации, также иногда используемый в задачах регрессии. Это, наравне с деревом решений, один из самых понятных подходов к классификации. На уровне интуиции суть метода такова: посмотри на соседей, какие преобладают, таков и ты. Формально основой метода является гипотеза компактности: если метрика расстояния между примерами введена достаточно удачно, то схожие примеры гораздо чаще лежат в одном классе, чем в разных.

Для классификации каждого из объектов тестовой выборки необходимо последовательно выполнить следующие операции:

Вычислить расстояние до каждого из объектов обучающей выборки
Отобрать k объектов обучающей выборки, расстояние до которых минимально
Класс классифицируемого объекта — это класс, наиболее часто встречающийся среди k ближайших соседей
Под задачу регрессии метод адаптируется довольно легко – на 3 шаге возвращается не метка, а число – среднее (или медианное) значение целевого признака среди соседей.

Примечательное свойство такого подхода – его ленивость. Это значит, что вычисления начинаются только в момент классификации тестового примера, а заранее, только при наличии обучающих примеров, никакая модель не строится. В этом отличие, например, от ранее рассмотренного дерева решений, где сначала на основе обучающей выборки строится дерево, а потом относительно быстро происходит классификация тестовых примеров.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
class KNeighborsClassifier:
    def __init__(self, n_neighbors=5):
        self.n_neighbors = n_neighbors

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        y_pred = []
        for i in range(len(X)):
            distances = np.sqrt(np.sum((self.X_train - X[i])**2, axis=1))
            nearest_indices = np.argsort(distances)[:self.n_neighbors]
            nearest_labels = self.y_train[nearest_indices]
            y_pred.append(np.bincount(nearest_labels).argmax())
        return np.array(y_pred)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
knn_y_train_pred = knn.predict(X_train)
knn_y_test_pred = knn.predict(X_test)

In [ ]:
accuracy_score(y_test, knn_y_test_pred)

0.8476190476190476

# Модель логистической регрессии

Логистическая регрессия является частным случаем линейного классификатора, но она обладает хорошим "умением" – прогнозировать вероятность отнесения наблюдения к классу. Таким образом, результат логистической регрессии всегда находится в интервале [0, 1].

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_y_train_pred = lr.predict(X_train)
lr_y_test_pred = lr.predict(X_test)

In [ ]:
accuracy_score(y_test, lr_y_test_pred)

0.8761904761904762

# Машина опорных векторов

Основная идея метода — перевод исходных векторов в пространство более высокой размерности и поиск разделяющей гиперплоскости с максимальным зазором в этом пространстве. Две параллельных гиперплоскости строятся по обеим сторонам гиперплоскости, разделяющей классы. Разделяющей гиперплоскостью будет гиперплоскость, максимизирующая расстояние до двух параллельных гиперплоскостей. Алгоритм работает в предположении, что чем больше разница или расстояние между этими параллельными гиперплоскостями, тем меньше будет средняя ошибка классификатора.

Kernel (ядро) отвечается за гиперплоскость и может принимать значения linear (для линейной), rbf (для нелинейной) и другие.


In [ ]:
from sklearn.svm import SVC

svm = SVC(kernel='linear')
svm.fit(X_train, y_train)
svm_y_train_pred = svm.predict(X_train)
svm_y_test_pred = svm.predict(X_test)

In [ ]:
accuracy_score(y_test, svm_y_test_pred)

0.9047619047619048